In [ ]:
---
comments: true
layout: post
title: Examples of Classes
description: Examples of classes in game
author: Alex Van Linge
permalink: /_notebooks/PBL_Blogs/2025-2-23-classes-examples.ipynb/
categories: [PBL Classes]
---

In this blog, I will use an excerpt from my `adventureGame` code to explain various aspects of classes like methods, instantiating objects, using objects, calling methods, parameters, and return values

## Excerpt from Player.js

In [ ]:
import GameEnv from './GameEnv.js';
import Character from './Character.js';
import Npc from './Npc.js'; // Import the Npc class
import Collectible from './Collectibles.js'; // Import the Collectible class
import Prompt from './Prompt.js'; // Import the Prompt module
import * as StatsManager from './StatsManager.js'; // Import the entire StatsManager module

class Player extends Character {
    constructor(data = null) {
        super(data);
        this.keypress = data?.keypress || { up: 87, left: 65, down: 83, right: 68 };
        this.velocity = { x: 0, y: 0 };
        this.position = {
            x: (GameEnv.innerWidth - (data?.pixels?.width || 0)) / 2 + 150, // Move the player 150 pixels to the right
            y: GameEnv.innerHeight - (data?.pixels?.height || 0)
        };
        this.yVelocity = 1.2 * (data?.yVelocity || 1); // Adjust the yVelocity to be slower than the current y-speed
        this.xVelocity = this.yVelocity; // Set the xVelocity to be the same as the yVelocity
        this.hasGreeted = false;
        this.bindEventListeners();
    }

    bindEventListeners() {
        addEventListener('keydown', this.handleKeyDown.bind(this));
        addEventListener('keyup', this.handleKeyUp.bind(this));
    }

    handleKeyDown({ keyCode }) {
        switch (keyCode) {
            case this.keypress.up:
                this.velocity.y = -this.yVelocity;
                this.direction = 'up';
                break;
            case this.keypress.left:
                this.velocity.x = -this.xVelocity;
                this.direction = 'left';
                break;
            case this.keypress.down:
                this.velocity.y = this.yVelocity;
                this.direction = 'down';
                break;
            case this.keypress.right:
                this.velocity.x = this.xVelocity;
                this.direction = 'right';
                break;
        }
        console.log(`Key down: ${keyCode}, Velocity: ${JSON.stringify(this.velocity)}`);
    }

    handleKeyUp({ keyCode }) {
        switch (keyCode) {
            case this.keypress.up:
            case this.keypress.down:
                this.velocity.y = 0;
                break;
            case this.keypress.left:
            case this.keypress.right:
                this.velocity.x = 0;
                break;
        }
        console.log(`Key up: ${keyCode}, Velocity: ${JSON.stringify(this.velocity)}`);
    }

    update() {
        this.position.x += this.velocity.x;
        this.position.y += this.velocity.y;

        console.log(`Position: ${JSON.stringify(this.position)}, Velocity: ${JSON.stringify(this.velocity)}`);

        // Check collision with NPC only
        const npc = GameEnv.gameObjects.find(obj => obj instanceof Npc);
        if (npc && this.isCollidingWithArea(npc)) {
            if (!this.hasGreeted) {
                this.hasGreeted = true; // Prevents repeated triggers
                npc.shareGreeting();
            }
            // Stop movement upon collision
            this.position.x -= this.velocity.x;
            this.position.y -= this.velocity.y;
            this.velocity.x = 0;
            this.velocity.y = 0;
        } else {
            this.hasGreeted = false; // Reset when no longer colliding
        }

        // Check collision with defined collision areas
        if (GameEnv.collisionAreas) {
            for (const collisionArea of GameEnv.collisionAreas) {
                if (this.isCollidingWithArea(collisionArea)) {
                    // Stop movement upon collision
                    this.position.x -= this.velocity.x;
                    this.position.y -= this.velocity.y;
                    this.velocity.x = 0;
                    this.velocity.y = 0;
                    break;
                }
            }
        }

        // Check collision with collectibles
        const collectible = GameEnv.gameObjects.find(obj => obj instanceof Collectible && !obj.collected);
        if (collectible && this.isCollidingWithArea(collectible)) {
            collectible.collect();
            StatsManager.updateCollectiblesRemaining();
        }

        // Check if player is in the exact bottom right corner
        if (this.position.x + this.width >= GameEnv.innerWidth && this.position.y + this.height >= GameEnv.innerHeight) {
            Prompt.showCustomPrompt("You have reached the bottom right corner!");
        }

        if (this.position.x + this.width > GameEnv.innerWidth) {
            this.position.x = GameEnv.innerWidth - this.width;
            this.velocity.x = 0;
        }
        if (this.position.x < 0) {
            this.position.x = 0;
            this.velocity.x = 0;
        }
        if (this.position.y + this.height > GameEnv.innerHeight) {
            this.position.y = GameEnv.innerHeight - this.height;
            this.velocity.y = 0;
        }
        if (this.position.y < 0) {
            this.position.y = 0;
            this.velocity.y = 0;
        }

        // Call the draw method to render the player
        this.draw();
    }

    isCollidingWithArea(area) {
        return (
            this.position.x < area.position.x + area.width &&
            this.position.x + this.width > area.position.x &&
            this.position.y < area.position.y + area.height &&
            this.position.y + this.height > area.position.y
        );
    }
}

export default Player;

### Explanation of Code 

Whoo! That was a lot of code, but thankfully this file involves a whole lot of class related stuff, so lets dive in shall we!


Methods:

Methods are functions defined within a class that operate on instances of that class. Examples of methods in the Player class include constructor, bindEventListeners, handleKeyDown, handleKeyUp, update, and isCollidingWithArea.

Instantiating Objects:

Objects are instances of classes created using the new keyword. In the Player class, an instance of Player is created with new Player(data).

Using Objects:

Objects are used to store data and provide methods to manipulate that data. In the Player class, the `this` keyword is used to refer to the current instance of the class.

Calling Methods:

Methods are called using the dot notation on an object. For example, `this.bindEventListeners()` calls the bindEventListeners method on the current instance of the Player class.

Parameters:

Parameters are values passed to methods to provide input. For example, the `handleKeyDown` method takes a parameter `{ keyCode }` to handle keydown events.

Return Values:

Methods can return values using the return statement. For example, the `isCollidingWithArea` method returns a boolean value indicating whether the player is colliding with a given area.


### How It Works


Constructor:

The constructor method initializes the Player object with default values and binds event listeners for keydown and keyup events.

Event Handling:

The bindEventListeners method adds event listeners for keydown and keyup events. The handleKeyDown and handleKeyUp methods update the player's velocity based on the key pressed or released.

Updating Position:

The update method updates the player's position based on the current velocity and checks for collisions with NPCs, defined collision areas, and collectibles.

Collision Detection:

The `isCollidingWithArea` method checks if the player is colliding with a given area by comparing the player's position and dimensions with the area's position and dimensions.

Rendering:

The draw method (not shown in the excerpt) would be responsible for rendering the player on the canvas.


This example demonstrates how the Player class in your adventure game uses methods, instantiates objects, calls methods, passes parameters, and returns values to manage the player's state and interactions within the game.

